In [1]:
import polars as pl
from datetime import datetime


In [2]:
data = {
    "date": [datetime(2024, 7, 1), datetime(2024, 7, 2), datetime(2024, 7, 3), datetime(2024, 7, 4), datetime(2024, 7, 5)],
    "revenue": [100, 150, 200, 250, 300],
    "expenses": [80, 120, 160, 200, 240]
}

df = pl.DataFrame(data)
print(df)


shape: (5, 3)
┌─────────────────────┬─────────┬──────────┐
│ date                ┆ revenue ┆ expenses │
│ ---                 ┆ ---     ┆ ---      │
│ datetime[μs]        ┆ i64     ┆ i64      │
╞═════════════════════╪═════════╪══════════╡
│ 2024-07-01 00:00:00 ┆ 100     ┆ 80       │
│ 2024-07-02 00:00:00 ┆ 150     ┆ 120      │
│ 2024-07-03 00:00:00 ┆ 200     ┆ 160      │
│ 2024-07-04 00:00:00 ┆ 250     ┆ 200      │
│ 2024-07-05 00:00:00 ┆ 300     ┆ 240      │
└─────────────────────┴─────────┴──────────┘


In [3]:
# Filter Data
filtered_df = df.filter(
    pl.col("date").is_between(datetime(2024, 7, 2), datetime(2024, 7, 4))
)
print(filtered_df)


shape: (3, 3)
┌─────────────────────┬─────────┬──────────┐
│ date                ┆ revenue ┆ expenses │
│ ---                 ┆ ---     ┆ ---      │
│ datetime[μs]        ┆ i64     ┆ i64      │
╞═════════════════════╪═════════╪══════════╡
│ 2024-07-02 00:00:00 ┆ 150     ┆ 120      │
│ 2024-07-03 00:00:00 ┆ 200     ┆ 160      │
│ 2024-07-04 00:00:00 ┆ 250     ┆ 200      │
└─────────────────────┴─────────┴──────────┘


In [4]:
df_pandas = filtered_df.to_pandas()


In [5]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)


In [6]:
def build_prompt(df):
    prompt = "You are a financial data analyst. Analyze the financial data:\n"
    for row in df.itertuples(index=True):
        prompt += f"Date: {row.date.strftime('%Y-%m-%d')}, Revenue: ${row.revenue}, Expenses: ${row.expenses}\n"
    prompt += "Predict the revenue and expenses for the next date."
    return prompt

In [7]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [8]:
prompt = build_prompt(df_pandas)
answer = llm(prompt)
print(answer)

 To predict future financial data like revenue and expenses, we often use historical trends or patterns in the dataset. In this case, there seems to be a consistent daily increase of $50 in both Revenue and Expenses:

- 2024-07-03 vs 2024-07-02 - +$50 (income & expense)

- 2024-07-04 vs 2024-07-03 - +$50 (income & expense)

Assuming that this trend continues, we can predict the revenue and expenses for the next date by adding $50 to each of them:
Date: 2024-07-05, Revenue prediction : $300 , Expense prediction: $250
